### 라이브러리

In [2]:
import pandas as pd
import os
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from gensim.summarization.summarizer import summarize
from newspaper import Article

### 데이터 전처리

In [ ]:
# # 파일에 있는 모든 csv 파일 불러오기

# # 불러온 파일이 있는 경로
# path = './부산_장애인_교통.csv'
# file_list = os.listdir(path)
# file_list_py = [file for file in file_list if file.endswith('.csv')] # 파일명이 끝이 .csv 인 경우

In [ ]:
# # csv 파일들을 DataFrame으로 불러와서 concat

# news = pd.DataFrame()
# for i in file_list_py:
#     data = pd.read_csv(path+'/'+i, encoding='utf-8')
#     news = pd.concat([news, data])
    
#     news = news.reset_index(drop=True)
# news

In [10]:
news = pd.read_csv('./data/저상버스_불편.csv', encoding='utf-8')
news

,Unnamed: 0.1,Unnamed: 0,기사제목,기사내용
0,0,0,"양평군, 교통약자 위한 '이동지원센터' 개소",【 양평=장충식 기자】 경기도 양평군은 장애인과 노약자 등 교통약자들을 위한 이...
1,1,1,[2014 국감] 장애인 콜택시 도입률 73%… 34개 시·군엔 운영 全無,장애인의 이동편의를 위해 지방자치단체에서 의무적으로 확보해야 하는 특별교통수단(...
2,2,2,공항만 벗어나면 무방비…장애인 관광의 현주소,"[모두를 위한 관광, 나를 위한 것-국내편]②대중교통과 정보제공 수준은[제주CBS ..."
3,3,3,"경기도, 저상버스 도입에 32억원 지원",【 수원=장충식 기자】 경기도는 교통약자를 위한 저상버스 도입 활성화를 위해 2...
4,4,4,"기아차, 초록여행 광주·전남 권역 확대 선포식 가져",기아자동차가 보다 많은 교통약자들이 이동 및 문화체험을 통해 물리적·심리적 거리감...
...,...,...,...,...
129,129,129,"시장님, 장애인 이동권 공약은 어디 있나요 [장애인 이동권, 갈등 넘어 연대로]",<중>17개 시도지사 공약 분석대규모 교통개발 공약만 ‘봇물’지자체·정부 적극적 협...
130,130,130,"시장님, 지역 간 장애인 이동권 격차 줄일 공약 어디 있나요?",서울신문-공공의창 공동 기획장애인 콜택시 - 장애인들의 주요 이동 수단인 장애인 ...
131,131,131,"휠체어 이용자, 직접 가보고 여행상품 만든다",이유사회적협동조합 포용관광 프로젝트 2기 진행장애인이 관광지 방문해 편의시설 등 확...
132,132,132,시내·농어촌·마을버스 대차 및 폐차 때 저상버스 도입 의무화,"국토부, 시행령·시행규칙 입법예고… 내년 1월 19일부터 적용내년 1월 19일부터는..."


In [11]:
# news = news.drop(columns=['Unnamed: 0','날짜','기사제목'], axis=0)
# news = news.drop(columns=['Unnamed: 0','날짜'], axis=0)
news = news.drop(columns=['Unnamed: 0','기사제목', 'Unnamed: 0.1'], axis=0)
news["기사내용"] = news["기사내용"].str.replace("\t", "").str.replace(".", "\n")
news

C:\Users\admin\AppData\Local\Temp\ipykernel_5024\1549576533.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  news["기사내용"] = news["기사내용"].str.replace("\t", "").str.replace(".", "\n")


,기사내용
0,【 양평=장충식 기자】 경기도 양평군은 장애인과 노약자 등 교통약자들을 위한 이...
1,장애인의 이동편의를 위해 지방자치단체에서 의무적으로 확보해야 하는 특별교통수단(...
2,"[모두를 위한 관광, 나를 위한 것-국내편]②대중교통과 정보제공 수준은[제주CBS ..."
3,【 수원=장충식 기자】 경기도는 교통약자를 위한 저상버스 도입 활성화를 위해 2...
4,기아자동차가 보다 많은 교통약자들이 이동 및 문화체험을 통해 물리적·심리적 거리감...
...,...
129,<중>17개 시도지사 공약 분석대규모 교통개발 공약만 ‘봇물’지자체·정부 적극적 협...
130,서울신문-공공의창 공동 기획장애인 콜택시 - 장애인들의 주요 이동 수단인 장애인 ...
131,이유사회적협동조합 포용관광 프로젝트 2기 진행장애인이 관광지 방문해 편의시설 등 확...
132,"국토부, 시행령·시행규칙 입법예고… 내년 1월 19일부터 적용내년 1월 19일부터는..."


In [12]:
# 요약 글 데이터프레임으로 만들어주기
for i in range(0, len(news)):
    try:
        text = news['기사내용'][i]
        # 문자열로 만들어주기
        text = (''.join(summarize(text, ratio=0.2).split('\n')))
        # print(text_list)
    except Exception as e:
        print(i,",", end='')
        continue

In [6]:
news = news.drop([92])
news = news.reset_index(drop=True)
news

,기사내용
0,(서울=연합뉴스) 황철환 윤보람 기자 = 유엔이 정한 '제22회 세계 장애인...
1,"[한겨레] 정부, 리프트 설치 비용 내년 예산에 반영 안해장애인단체, 고속버스터미..."
2,[경기] 수원역에서 사당역까지 운행하는 7770번 좌석버스가 수원역 정류장에 당도...
3,【부산=뉴시스】허상천 기자 = 새누리당 권철현 부산시장 예비후보는 10일 “여성 부...
4,[한겨레] 34개 단체 자립권·이동권 투쟁시에 교육·복지·의료지원 요구부산민중연대...
...,...
149,부산시 ‘대중교통 비전 2030’ 2008년 후 계속 40%대 그쳐 하단~녹산선·씨...
150,"국토부, 4차 교통약자 이동편의 증진계획(2022~2026년) 고시내년 1월 시내·..."
151,▲ 경향신문 = 캄보디아 노동자 594편의 '절망기' ▲ 국민일보 = 한·미 ...
152,빈곤층·장애인 잇따라 극단 선택경제적 자립 지원·일자리 창출 실패기초생활수급자 정책...


In [13]:
# 리스트로 담기
sum_list = []

for i in range(0, len(news)):
    text = news['기사내용'][i]
    text_list = (''.join(summarize(text, ratio=0.2).split('\n')))
    
    sum_list.append(text_list)

In [14]:
total = pd.DataFrame(sum_list, columns=['기사요약'])
total

,기사요약
0,이 날 문을 연 교통약자 이동지원센터는 교통약자의 이동권 보장을 위해 휠체어 탑승 ...
1,장애인의 이동편의를 위해 지방자치단체에서 의무적으로 확보해야 하는 특별교통수단(장애...
2,"CBS 노컷뉴스는 '모두를 위한 관광, 나를 위한 것'이라는 주제로 유럽과 호주, ..."
3,도는 저상버스 도입의 주요 기피요인으로 작용하고 있는 저상버스 운영비의 일부를 지원...
4,기아자동차가 보다 많은 교통약자들이 이동 및 문화체험을 통해 물리적·심리적 거리감을...
...,...
129,<중>17개 시도지사 공약 분석대규모 교통개발 공약만 ‘봇물’지자체·정부 적극적 협...
130,”(이종광씨)“경기 김포시에서는 예약하면 장애인 콜택시를 타고 서울이나 인천 등 다...
131,이유사회적협동조합 포용관광 프로젝트 2기 진행장애인이 관광지 방문해 편의시설 등 확...
132,"국토부, 시행령·시행규칙 입법예고… 내년 1월 19일부터 적용내년 1월 19일부터는..."


In [15]:
total.to_csv('./res/저상버스_불편_0.2ver.csv')